
# Step 1 - Create Training Dataset

In [ ]:
# Load libraries
import cv2
import numpy as np
import os

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face


def user_capture(user):

    # Initialize Webcam
    cap = cv2.VideoCapture(0)
    count = 0
    
    #Create faces folder to put dataset
    os.system('mkdir .\\faces')
    #Create folder with user's name
    os.system('mkdir .\\faces\{}'.format(user))

    # Collect 100 samples of your face from webcam input
    while True:

        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200)) #resize to (200, 200)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Save file in specified directory with unique name
            file_name_path = './faces/{}/'.format(user) + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)

            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)

        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break
    cap.release()
    cv2.destroyAllWindows()


users = []
models = []
no_users = int(input("Enter the number of users:- "))
for i in range(no_users):
    user =  input("Enter user's name:- ")
    users.append(user)
    user_capture(user)

In [30]:
print("----------------")
print("Images for training collected successfully!!")
print("----------------")

----------------
Images for training collected successfully!!
----------------


# Step 2 - Training the Model

In [3]:
# pip install opencv-contrib-python
def train_model(users):
    
    import cv2
    import numpy as np
    from os import listdir
    from os.path import isfile, join
    
    for c in range(len(users)):
        
        # Get the training data we previously made
        data_path = './faces/{}/'.format(users[c])
        onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

        # Create arrays for training data and labels
        Training_Data, Labels = [], []

        # Open training images in our datapath
        # Create a numpy array for training data
        for i, files in enumerate(onlyfiles):
            image_path = data_path + onlyfiles[i]
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)

        # Create a numpy array for both training data and labels
        Labels = np.asarray(Labels, dtype=np.int32)

        # Initialize facial recognizer
        # model = cv2.face.createLBPHFaceRecognizer()
        # NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
        # pip install opencv-contrib-python
        # model = cv2.createLBPHFaceRecognizer()

        models.append(cv2.face_LBPHFaceRecognizer.create())
        # Let's train our model 
        models[c].train(np.asarray(Training_Data), np.asarray(Labels))
        print("Model trained sucessefully for:- "+ users[c])

In [4]:
users

['sam', 'ojas', 'tina']

In [5]:
train_model(users)

Model trained sucessefully for:- sam
Model trained sucessefully for:- ojas
Model trained sucessefully for:- tina


In [6]:
models

[<face_LBPHFaceRecognizer 00000208FF5B27B0>,
 <face_LBPHFaceRecognizer 00000208FF5B2810>,
 <face_LBPHFaceRecognizer 00000208FF5B27F0>]

# Function for Sending Mail

In [7]:
def mail(pred_user):
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    mail_content = '''Hello,\n
    This is Your system. \n
    This is face of {} !! \n
    Thank You.
    '''.format(pred_user)
    
    #The mail addresses and password
    sender_address = 'aa0413371'
    sender_pass = '*************'
    receiver_address = 'samriddhi2958@gmail.com'
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'A test mail sent by Python. It has an attachment.'
    #The subject line
    #The body and the attachments for the mail
    #message.attach(MIMEText(mail_content, 'plain'))
    message.attach(MIMEText(mail_content, 'html'))
    attach_file_name = 'sam.jpg'
    attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
    payload = MIMEBase('application', 'octate-stream')
    payload.set_payload((attach_file).read())
    encoders.encode_base64(payload) #encode the attachment
    #add payload header with filename
    payload.add_header("Content-Disposition", f"attachment; filename= {attach_file_name}",)
    message.attach(payload)
    
    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent!!')

# Function for Sending Whatsapp Message

In [8]:
# Setup twillio account and twilio sandbox
# pip install twilio
def whatsapp():
    
    import os
    os.system('pip install twilio')
    
    import os
    from twilio.rest import Client

    account_sid = 'AC**********b4ac2'
    auth_token = '63*******************c97'
    client = Client(account_sid, auth_token)

    from_whatsapp_number = 'whatsapp:+14*******'
    to_whatsapp_number = 'whatsapp:*******'

    message = client.messages.create(body='Somebody is here!!',
                           media_url='https://cdn3.vectorstock.com/i/1000x1000/52/02/smiling-cartoon-hacker-holding-a-hacked-smartphone-vector-15735202.jpg',
                           from_=from_whatsapp_number,
                           to=to_whatsapp_number)

    print("Whatsapp message sent!!")
    print("Sender's id:- " , message.sid)

# Launch EC2 instance, create EBS volume and attach to instance

In [9]:
#aws configure 
# run in command prompt, to give secret and access key
def aws_ec2_ebs():
    import os

    key_name = input("Enter Key Name:- ")
    az = input("Enter AZ Name:- ")
    subnet = input("Enter Subnet ID:- ")
    os.system('aws ec2 run-instances --image-id ami-0aeeebd8d2ab47354 --instance-type t2.micro --key-name '+ key_name + ' --subnet-id '+ subnet + ' --tag-specifications "ResourceType = instance , Tags = [{Key="Name",Value="face_recog"}]"')
    print("Launched EC2 instance..")
    os.system('aws ec2 create-volume --availability-zone ' + az +' --size 5')
    print("Created EBS volume..")
    inst_id = input("Enter Instance Id:- ")
    vol_id = input("Enter Volume Id:- ")
    dev_name = input("Enter Device Name:- ")
    os.system('aws ec2 attach-volume --instance-id '+inst_id+' --volume-id '+ vol_id+' --device '+ dev_name)
    print("Attached EBS volume to EC2 instance!!")
    

# Step 3 - Run Facial Recognition

In [25]:
import cv2
import numpy as np
import os
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0:
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


def get_confidence_score(results):
    if results[1] < 500:
        confidence = int( 100 * (1 - (results[1])/400) )
    return confidence


def face_recog():
    # Open Webcam
    cap = cv2.VideoCapture(0)

    while True:

        ret, frame = cap.read()
        image, face = face_detector(frame)

        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Pass face to prediction model
            # "results" comprises of a tuple containing the label and the confidence value
            results = []
            confidences = []
            c = 0

            for c in range(len(users)):
                results.append(models[c].predict(face))

            for c in range(len(users)):
                confidences.append(get_confidence_score(results[c]))

            pred_user  = None

            largest = max(confidences)
            conf_ind = confidences.index(largest)
            confidence = confidences[conf_ind]
            pred_user = users[conf_ind]

            display_string = str(confidence) + '% Confident it is ' + pred_user
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)


            if confidence >= 91:
                cv2.putText(image, "Hi!! "+ pred_user, (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition', image )
                if pred_user == "sam":
                    cv2.imwrite("{}.jpg".format(pred_user), image)
                    cv2.imshow('Face Recognition', image )
                    cv2.waitKey(2)
                    cv2.destroyAllWindows()
                    mail(pred_user)
                    whatsapp()
                    break
                elif pred_user == "ojas":
                    cv2.imshow('Face Recognition', image )
                    cv2.waitKey(2)
                    cv2.destroyAllWindows()
                    aws_ec2_ebs()
                    break
                else:
                    cv2.imshow('Face Recognition', image )
                    cv2.waitKey(2)
                    cv2.destroyAllWindows()
                    print("Hello {}!!!!!!!".format("tina"))
                    break
                break
            else:   
                #cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )

        except:
            cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            pass

        if cv2.waitKey(10) == 13: #13 is the Enter Key
            break

    cap.release()
    cv2.destroyAllWindows() 
    print()
    print("----------------")
    print("All completed!!!")
    print("----------------")

In [29]:
face_recog()

Enter Key Name:- arthos
Enter AZ Name:- us-east-1a
Enter Subnet ID:- subnet-0c5a276a
Launched EC2 instance..
Created EBS volume..
Enter Instance Id:- i-09920fea663927c4f
Enter Volume Id:- vol-0681777b03981f2a0
Enter Device Name:- /dev/xvde
Attached EBS volume to EC2 instance!!

----------------
All completed!!!
----------------
